In [16]:
import spacy
import random
import pandas as pd
import json
import re
from urllib.request import urljoin 
from bs4 import BeautifulSoup 
import requests 
from urllib.request import urlparse 
from urllib.error import HTTPError
from urllib.error import URLError
import openpyxl
from openpyxl import Workbook
from openpyxl.styles import Color, PatternFill, Font, Border
from openpyxl.styles import colors
from openpyxl.cell import Cell
from openpyxl.styles import Color, Fill
from openpyxl.cell import Cell

In [ ]:
from google.colab import drive
drive.mount("my-drive1")

Drive already mounted at my-drive; to attempt to forcibly remount, call drive.mount("my-drive", force_remount=True).


In [17]:
nlp_model=spacy.load('my-drive/My Drive/nlp_model')

In [18]:
def selected_tags(tag):
  return tag.name!='a' and tag.name!='style' and tag.name!='script' and tag.name!='title'
def extract_info(url):
  try:
    page=requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    content=soup.find_all(selected_tags)
    all_links=soup.find_all('a')
    contentstr=', '.join([str(alltext.text).strip() for alltext in content if '\n' not in alltext.text])
    email=set()
    personal_website=set()
    print('Scraping Url: '+url)
    for link in all_links:
      if '@' in link.text and '.' in link.text:
        mail=link.text.strip()
        if 'mailto:' in mail:
          mail=mail.replace('mailto:','')
        elif 'mailto' in mail:
          mail=mail.replace('mailto','')
        email.add(mail)
      elif 'email' in str(link.text).lower():
        mail=link.get('href').strip()
        if 'mailto:' in mail:
          mail=mail.replace('mailto:','')
        elif 'mailto' in mail:
          mail=mail.replace('mailto','')
        email.add(mail)
        print(link.text+'\t'+link.get('href'))
      if 'website' in str(link.text).lower() and (('https' in link.get('href')) or ('http' in link.get('href'))):
        personal_website.add(link.get('href').strip())
        print(link.text+'\t'+link.get('href'))
      elif 'profile' in str(link.text).lower() and (('https' in link.get('href')) or ('http' in link.get('href'))):
        personal_website.add(link.get('href').strip())
        print(link.text+'\t'+link.get('href'))
    return contentstr,list(email),list(personal_website)
  except HTTPError as e:
    print("404 Not Found")
    return "",[],[]
  except URLError as e:
    print("The server can't be reached")
    return "",[],[]
  except:
    print("Some unexpected error occurred")
    return "",[],[]

In [19]:
df = pd.read_excel("my-drive/My Drive/surnames_indian.xlsx")
last_names = df['Names'].tolist()
last_names = [ele.lower() for ele in last_names]
last_names = set(last_names)

df1 = pd.read_excel("my-drive/My Drive/indian_first_names.xlsx")
first_names = df1['Names'].tolist()
first_names = [ele.lower() for ele in first_names]
first_names = set(first_names)

dictionary = pd.read_excel("my-drive/My Drive/dictionary.xlsx")
dic_keys = set(dictionary['Dictionary'].tolist())

def is_dict_words(text):
  text = text.lower()
  text_list = text.split(" ")
  for ele in text_list:
    if ele in dic_keys:
      return True
  return False

In [20]:
def is_indian_name(full_name):
  full_name = full_name.lower()
  name_list = full_name.split(" ")
  first_name = name_list[0]
  # last_name = name_list[-1]
  if len(name_list)==1:
    if first_name in first_names:
      return True
    return False
  else:
    for ele in name_list:
      if ele in last_names:
        return True
  return False


def is_faculty_page(url):
  try:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    names_cnt = 0
    all_links = soup.find_all('a')
    for link in all_links:
      result = is_indian_name(link.text)
      if result:
        names_cnt = names_cnt+1
    
    if names_cnt>=1:
      return True
    return False
  except HTTPError as e:
    print("404 Not Found")
    return None
  except URLError as e:
    print("The server can't be reached")
    return None
  except:
    print("Some unexpected error occurred")
    return None

In [21]:
def tentative_homepage_urls_adder(url,MAX_DEPTH,base_url):

  faculty_keys=['faculty','professor','researcher','doctorate','teacher','lecturer','assistant','people','staff','affiliates']
  pending = list()
  pending.append(url)
  visited=set()
  depth = 0
  ten_fauclty_urls = set()

  while(depth<MAX_DEPTH and len(pending)!=0):
    print("At depth "+ str(depth))
    size=len(pending)
    depth=depth+1

    while(size!=0):
      size=size-1
      link = pending.pop(0)
      if link in visited:
        continue
      visited.add(link)
      print(link)
      
      try:
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        all_links = soup.find_all('a')
        ten_fauclty_urls.add(link)
      except HTTPError as e:
        print("404 Not Found")
        continue
      except URLError as e:
        print("The server can't be reached")
        continue
      except:
        print("Some unexpected error occurred")
        continue
      for li in all_links:
        res = False
        for ele in faculty_keys:
          condition = str(li.text)
          if ele in condition.lower():
            res = True
            
        if res and li.get('href')!=None and len(li.get('href'))!=0:
          if "http" not in li.get('href'):
            if li.get('href') not in visited:
              pending.append(base_url+str(li.get('href')))
          else:
            if li.get('href') not in visited:
              pending.append(li.get('href')) 

  return ten_fauclty_urls

In [22]:
def find_new_faculty_info(path,content_info):
  newdf_dict={'Name':[],'Specialization':[],'Department':[],'Designation':[],'Address':[],'Qualification':[],'Experience':[],'Degree':[],'Email':[],'PersonalWebsite':[],'MobileNumber':[],'URL':[]}
  for i in range(len(content_info)):
    Experience='Not Found'
    Address='Not Found'
    Specialization='Not Found'
    Email='Not Found'
    Designation='Not Found'
    Qualification='Not Found'
    Degree='Not Found'
    MobileNumber='Not Found'
    PersonalWebsite='Not Found'
    Department='Not Found'
    doc=nlp_model(content_info.at[i,'content'])
    for ent in doc.ents:
      if ent.label_=='Experience':
        if Experience!='Not Found':
          Experience=Experience+", "+ent.text
        else:
          Experience=ent.text
      elif ent.label_=='Address':
        if Address=='Not Found':
          Address=ent.text
      elif ent.label_=='Specialization':
        if Specialization!='Not Found':
          Specialization=Specialization+", "+ent.text
        else:
          Specialization=ent.text
      elif ent.label_=='Designation':
        if Designation=='Not Found':
          Designation=ent.text
      elif ent.label_=='Qualification':
        if Qualification!='Not Found':
          Qualification=Qualification+", "+ent.text
        else:
          Qualification=ent.text
      elif ent.label_=='Degree':
        if Degree!='Not Found':
          Degree=Degree+", "+ent.text
        else:
          Degree=ent.text
      elif ent.label_=='MobileNumber':
        if MobileNumber=='Not Found':
          MobileNumber=ent.text
      elif ent.label_=='Department':
        if Department=='Not Found':
          Department=ent.text
      print(ent.text+"\t"+ent.label_)
    if len(content_info.at[i,'email'])!=0:
      Email=content_info.at[i,'email']
    if len(content_info.at[i,'personal_website'])!=0:
      PersonalWebsite=content_info.at[i,'personal_website']
    newdf_dict['Address'].append(Address)
    newdf_dict['Degree'].append(Degree)
    newdf_dict['Department'].append(Department)
    newdf_dict['Designation'].append(Designation)
    newdf_dict['Email'].append(Email)
    newdf_dict['Experience'].append(Experience)
    newdf_dict['MobileNumber'].append(MobileNumber)
    newdf_dict['Name'].append(content_info.at[i,'name'])
    newdf_dict['PersonalWebsite'].append(PersonalWebsite)
    newdf_dict['Qualification'].append(Qualification)
    newdf_dict['Specialization'].append(Specialization)
    newdf_dict['URL'].append(content_info.at[i,'url'])

  newdatadf = pd.DataFrame.from_dict(newdf_dict)
  newdatadf.sort_values('Name',inplace=True)
  final_filtration(path, newdatadf)

In [23]:
def final_filtration(path, dataset):
  final_dict = dict()
  for r in range(len(dataset.index)):
    name = dataset.iloc[r][0]
    name = name.capitalize()
    if name in final_dict.keys():
      specialization = dataset.iloc[r][1]
      department = dataset.iloc[r][2]
      designation = dataset.iloc[r][3]
      address = dataset.iloc[r][4]
      qualification = dataset.iloc[r][5]
      experience = dataset.iloc[r][6]
      degree = dataset.iloc[r][7]
      email = dataset.iloc[r][8]
      personalWebsite = dataset.iloc[r][9]
      mobileNumber = dataset.iloc[r][10]
      URL = dataset.iloc[r][11]
      if specialization!="Not Found" and specialization not in final_dict[name]['specialization']:
        final_dict[name]['specialization'].append(str(specialization))
        final_dict[name]['specialization'] = [ele for ele in final_dict[name]['specialization'] if ele!="Not Found"]
      if department!="Not Found" and department not in final_dict[name]['department']:
        final_dict[name]['department'].append(str(department))
        final_dict[name]['department'] = [ele for ele in final_dict[name]['department'] if ele!="Not Found"]
      if designation!="Not Found" and designation not in final_dict[name]['designation']:
        final_dict[name]['designation'].append(str(designation))
        final_dict[name]['designation'] = [ele for ele in final_dict[name]['designation'] if ele!="Not Found"]
      if address!="Not Found" and address not in final_dict[name]['address']:
        final_dict[name]['address'].append(str(address))
        final_dict[name]['address'] = [ele for ele in final_dict[name]['address'] if ele!="Not Found"]
      if qualification!="Not Found" and qualification not in final_dict[name]['qualification']:
        final_dict[name]['qualification'].append(str(qualification))
        final_dict[name]['qualification'] = [ele for ele in final_dict[name]['qualification'] if ele!="Not Found"]
      if experience!="Not Found" and experience not in final_dict[name]['experience']:
        final_dict[name]['experience'].append(str(experience))
        final_dict[name]['experience'] = [ele for ele in final_dict[name]['experience'] if ele!="Not Found"]
      if degree!="Not Found" and degree not in final_dict[name]['degree']:
        final_dict[name]['degree'].append(str(degree))
        final_dict[name]['degree'] = [ele for ele in final_dict[name]['degree'] if ele!="Not Found"]
      if email!="Not Found" and email not in final_dict[name]['email']:
        final_dict[name]['email'].append(str(email))
        final_dict[name]['email'] = [ele for ele in final_dict[name]['email'] if ele!="Not Found"]
      if personalWebsite!="Not Found" and personalWebsite not in final_dict[name]['personalWebsite']:
        final_dict[name]['personalWebsite'].append(str(personalWebsite))
        final_dict[name]['personalWebsite'] = [ele for ele in final_dict[name]['personalWebsite'] if ele!="Not Found"]
      if mobileNumber!="Not Found" and mobileNumber not in final_dict[name]['mobileNumber']:
        final_dict[name]['mobileNumber'].append(str(mobileNumber))
        final_dict[name]['mobileNumber'] = [ele for ele in final_dict[name]['mobileNumber'] if ele!="Not Found"]
      if URL!="Not Found" and URL not in final_dict[name]['URL']:
        final_dict[name]['URL'].append(str(URL))
        final_dict[name]['URL'] = [ele for ele in final_dict[name]['URL'] if ele!="Not Found"]
    else:
      specialization = dataset.iloc[r,1]
      department = dataset.iloc[r,2]
      designation = dataset.iloc[r,3]
      address = dataset.iloc[r,4]
      qualification = dataset.iloc[r,5]
      experience = dataset.iloc[r,6]
      degree = dataset.iloc[r,7]
      email = dataset.iloc[r,8]
      personalWebsite = dataset.iloc[r,9]
      mobileNumber = dataset.iloc[r,10]
      URL = dataset.iloc[r,11]
      final_dict[name] = dict()
      final_dict[name]['specialization'] = []
      final_dict[name]['department'] = []
      final_dict[name]['designation'] = []
      final_dict[name]['address'] = []
      final_dict[name]['qualification'] = []
      final_dict[name]['experience'] = []
      final_dict[name]['degree'] = []
      final_dict[name]['email'] = []
      final_dict[name]['personalWebsite'] = []
      final_dict[name]['mobileNumber'] = []
      final_dict[name]['URL'] = []

      final_dict[name]['specialization'].append(specialization)
      final_dict[name]['department'].append(department)
      final_dict[name]['designation'].append(designation)
      final_dict[name]['address'].append(address)
      final_dict[name]['qualification'].append(qualification)
      final_dict[name]['experience'].append(experience)
      final_dict[name]['degree'].append(degree)
      final_dict[name]['email'].append(email)
      final_dict[name]['personalWebsite'].append(personalWebsite)
      final_dict[name]['mobileNumber'].append(mobileNumber)
      final_dict[name]['URL'].append(URL)

  resultant_dict={'Name':[],'Specialization':[],'Department':[],'Designation':[],'Address':[],'Qualification':[],'Experience':[],'Degree':[],'Email':[],'PersonalWebsite':[],'MobileNumber':[],'URL':[]}
  for key,value in final_dict.items():
    resultant_dict['Name'].append(key)
    resultant_dict['Address'].append(' '.join(value['address']))
    resultant_dict['Degree'].append(' '.join(value['degree']))
    resultant_dict['Department'].append(' '.join(value['department']))
    resultant_dict['Designation'].append(' '.join(value['designation']))
    resultant_dict['Email'].append(' '.join(value['email']))
    resultant_dict['Experience'].append(' '.join(value['experience']))
    resultant_dict['MobileNumber'].append(' '.join(value['mobileNumber']))
    resultant_dict['PersonalWebsite'].append(' '.join(value['personalWebsite']))
    resultant_dict['Qualification'].append(' '.join(value['qualification']))
    resultant_dict['Specialization'].append(' '.join(value['specialization']))
    resultant_dict['URL'].append(', '.join(value['URL']))

  result_df = pd.DataFrame.from_dict(resultant_dict)
  if '.csv' in path:
    path = path.replace('.csv','_result_new.xlsx')
  elif '.xlsx' in path:
    path=path.replace('.xlsx','_result_new.xlsx')

  result_df.sort_values('Name',inplace=True)
  result_df.drop_duplicates(inplace = True)
  result_df.to_excel(path,index=False)
    

In [24]:
def add_new_faculties(path,faculty_dict):
  faculty_list=[[url,name] for url,name in faculty_dict.items()]
  df=pd.DataFrame(faculty_list,columns=['url','name'])
  info_dict={'name':[],'content':[],'email':[],'personal_website':[],'url':[]}
  for i in range(len(df)):
    content,email,personal_website=extract_info(df.at[i,'url'])      
    email=set(email)
    email=list(email)
    personal_website=set(personal_website)
    personal_website=list(personal_website)
    email_str=', '.join([str(everymail) for everymail in email])
    personal_website_str=', '.join([str(everywebsite) for everywebsite in personal_website])
    print('Email: '+email_str)
    print('Website: '+personal_website_str)
    if len(content)>1000000:
      content=content[:900000]
    info_dict['content'].append(content.strip())
    info_dict['email'].append(email_str.strip())
    info_dict['personal_website'].append(personal_website_str.strip())
    info_dict['name'].append(df.at[i,'name'])
    info_dict['url'].append(df.at[i,'url'])
  content_df=pd.DataFrame.from_dict(info_dict)
  content_df.sort_values('name')
  find_new_faculty_info(path,content_df)

In [25]:
def find_new_faculties(path,faculty_homepage_urls,original_urls):
  indian_names_dict = dict()

  for link in faculty_homepage_urls:
    try:
      page = requests.get(link)
      soup = BeautifulSoup(page.content, 'html.parser')
      all_links = soup.find_all('a')
    except HTTPError as e:
      print("404 Not Found")
      continue
    except URLError as e:
      print("The server can't be reached")
      continue
    except:
      print("Some unexpected error occurred")
      continue
    first_id = link.find('/')
    second_id = link.find('/',first_id+1)
    third_id = link.find('/',second_id+1)
    base_url = link[:third_id]
    for every_link in all_links:
      if is_indian_name(every_link.text.strip()):
        absolute_url = every_link.get('href')
        if "http" not in every_link.get('href'):
          absolute_url = base_url + every_link.get('href')
        if is_dict_words(every_link.text.strip()) == False:
          new_url = absolute_url
          if 'http:' in new_url:
            new_url=new_url.replace('http:','https:')
          indian_names_dict[new_url] = every_link.text.strip()

  new_faculty_urls = []
  for key in indian_names_dict.keys():
    if key not in original_urls:
      new_faculty_urls.append(key)

  updated_indian_faculty_dict = dict()
  for key in new_faculty_urls:
    updated_indian_faculty_dict[key] = indian_names_dict[key]

  for key, value in updated_indian_faculty_dict.items():
    print(key+" -- "+value)

  add_new_faculties(path,updated_indian_faculty_dict)

In [26]:
def find_tentative_urls(path,df):
  urls=list(df['URL'])
  original_urls=set()
  univ_names={}
  for url_str in urls:
    url_list=url_str.split(',')
    for url in url_list:
      if '.edu' in url:
        if 'http:' in url:
          url=url.replace('http:','https:')
        end=url.find('.edu')
        start=url.rfind('.',0,end-1)
        univ=url[start+1:end]
        if univ in univ_names.keys():
          univ_names[univ]=univ_names[univ]+1
        else:
          univ_names[univ]=1
        original_urls.add(url)
  original_urls=list(original_urls)
  tentative_urls=set()
  home_urls=set()
  univ_name=max(univ_names,key=univ_names.get)
  for url in original_urls:
    first_id = url.find('/')
    second_id = url.find('/',first_id+1)
    third_id = url.find('/',second_id+1)
    url = url[:third_id]
    if univ_name in url:
      home_urls.add(url)

  for url in home_urls:
    return_list = tentative_homepage_urls_adder(url,3,url)
    tentative_urls=list(tentative_urls)
    tentative_urls.extend(return_list)
    tentative_urls = set(tentative_urls)

  directory_urls=set()
  for url in original_urls:
    link = url[:url.rindex('/',0,len(url)-2)]
    if univ_name in url and link not in home_urls:
      directory_urls.add(link)

  for url in directory_urls:
    first_id = url.find('/')
    second_id = url.find('/',first_id+1)
    third_id = url.find('/',second_id+1)
    base_url = url[:third_id]
    return_list = tentative_homepage_urls_adder(url,2,base_url)
    tentative_urls = list(tentative_urls)
    tentative_urls.extend(return_list)
    tentative_urls = set(tentative_urls)

  print(tentative_urls)
  faculty_homepage_urls = set(tentative_urls)

  # for url in tentative_urls:
  #   if is_faculty_page(url):
  #     faculty_homepage_urls.add(url)
  find_new_faculties(path,faculty_homepage_urls,original_urls)

In [27]:
def read_file(path):
  df=''
  if path.endswith('.csv'):
    df=pd.read_csv(path)
  elif path.endswith('.xlsx'):
    df=pd.read_excel(path)
  else:
    print('This file format is not accepted')
    return
  find_tentative_urls(path,df)

In [28]:
print('Enter name of file:')
filename=input()
basepath='my-drive/My Drive/'
read_file(basepath+filename)

Enter name of file:
1 california institute of technology.xlsx
At depth 0
https://chemistry.caltech.edu
Some unexpected error occurred
At depth 0
https://www.gps.caltech.edu
At depth 1
https://www.gps.caltech.edu/resources/quick-links-1/quick-links-faculty
https://www.gps.caltech.edu/resources/quick-links-1/quick-links-staff
https://www.gps.caltech.edu/people
https://www.gps.caltech.edu/people?category=17
https://www.gps.caltech.edu/people?category=24
https://www.gps.caltech.edu/people?category=29
https://www.gps.caltech.edu/people?category=8
https://www.gps.caltech.edu/people?category=14
https://www.gps.caltech.edu/people?category=16
https://www.gps.caltech.edu/about/positions-available/faculty-positions
https://www.gps.caltech.edu/about/positions-available/staff-positions
At depth 2
http://www.oof.caltech.edu/
At depth 0
https://nanofab.caltech.edu
Some unexpected error occurred
At depth 0
https://caltech.academia.edu
At depth 1
http://www.academia.edu/Directory/People
https://caltech